<a href="https://colab.research.google.com/github/Mohith-11/Plant-disease-detection/blob/main/plant_disease_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import filesuploaded = files.upload()       

In [ ]:
!rm -rf dataset!unzip -q "New Plant Diseases Dataset(Augmented).zip" -d dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGeneratordata_dir = '/content/dataset/New Plant Diseases Dataset(Augmented)/train'datagen = ImageDataGenerator(    rescale=1./255,    rotation_range=25,    zoom_range=0.2,    shear_range=0.1,    horizontal_flip=True,    validation_split=0.2   )train_gen = datagen.flow_from_directory(    data_dir,    target_size=(128,128),    batch_size=32,    class_mode='categorical',    subset='training')val_gen = datagen.flow_from_directory(    data_dir,    target_size=(128,128),    batch_size=32,    class_mode='categorical',    subset='validation')

In [ ]:
from tensorflow.keras import layers, modelsmodel = models.Sequential([    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),    layers.MaxPooling2D(2,2),    layers.Conv2D(64, (3,3), activation='relu'),    layers.MaxPooling2D(2,2),    layers.Conv2D(128, (3,3), activation='relu'),    layers.MaxPooling2D(2,2),    layers.Flatten(),    layers.Dense(128, activation='relu'),    layers.Dropout(0.3),    layers.Dense(train_gen.num_classes, activation='softmax')])model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])model.summary()

In [ ]:
history = model.fit(train_gen, validation_data=val_gen, epochs=15)

In [ ]:
import matplotlib.pyplot as pltplt.plot(history.history['accuracy'], label='Train Accuracy')plt.plot(history.history['val_accuracy'], label='Validation Accuracy')plt.legend()plt.title('Training vs Validation Accuracy')plt.show()plt.plot(history.history['loss'], label='Train Loss')plt.plot(history.history['val_loss'], label='Validation Loss')plt.legend()plt.title('Training vs Validation Loss')plt.show()

In [ ]:
model.save('cnn_model_v1.h5')from google.colab import filesfiles.download('cnn_model_v1.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGeneratordata_dir = '/content/dataset/New Plant Diseases Dataset(Augmented)/train'datagen = ImageDataGenerator(    rescale=1./255,    rotation_range=25,    zoom_range=0.2,    shear_range=0.1,    horizontal_flip=True,    validation_split=0.2)train_gen = datagen.flow_from_directory(    data_dir,    target_size=(224,224),    batch_size=32,    class_mode='categorical',    subset='training')val_gen = datagen.flow_from_directory(    data_dir,    target_size=(224,224),    batch_size=32,    class_mode='categorical',    subset='validation')

In [ ]:
from tensorflow.keras.applications import MobileNetV2from tensorflow.keras import layers, modelsbase_model = MobileNetV2(    weights='imagenet',    include_top=False,    input_shape=(224,224,3))base_model.trainable = False  model = models.Sequential([    base_model,    layers.GlobalAveragePooling2D(),    layers.Dropout(0.3),    layers.Dense(train_gen.num_classes, activation='softmax')])model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStoppingearly_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)history = model.fit(    train_gen,    validation_data=val_gen,    epochs=10,    callbacks=[early_stop])

In [ ]:
import numpy as npfrom sklearn.metrics import confusion_matrix, classification_reportimport matplotlib.pyplot as pltimport seaborn as snsval_gen.reset()preds = model.predict(val_gen)y_pred = np.argmax(preds, axis=1)y_true = val_gen.classesprint(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))cm = confusion_matrix(y_true, y_pred)plt.figure(figsize=(8,6))sns.heatmap(cm, annot=False, cmap='Blues')plt.title('Confusion Matrix')plt.xlabel('Predicted')plt.ylabel('Actual')plt.savefig('confusion_matrix.png')plt.show()

In [ ]:
import randomfrom tensorflow.keras.preprocessing import imagefiles = val_gen.filepathssample_files = random.sample(files, 4)plt.figure(figsize=(8,8))for i, img_path in enumerate(sample_files):    img = image.load_img(img_path, target_size=(224,224))    x = image.img_to_array(img)/255.0    x = np.expand_dims(x, axis=0)    pred = model.predict(x)    class_name = list(val_gen.class_indices.keys())[np.argmax(pred)]    plt.subplot(2,2,i+1)    plt.imshow(img)    plt.title(f'Predicted: {class_name}')    plt.axis('off')plt.savefig('sample_predictions.png')plt.show()

In [ ]:
model.save('mobilenetv2_model.h5')from google.colab import filesfiles.download('mobilenetv2_model.h5')files.download('confusion_matrix.png')files.download('sample_predictions.png')

In [ ]:
import tensorflow as tfimport numpy as npimport matplotlib.pyplot as pltfrom tensorflow.keras.preprocessing import imageimport cv2import os

In [ ]:
model = tf.keras.models.load_model('/content/mobilenetv2_model.h5')print("Model loaded successfully!")

In [ ]:
IMG_SIZE = (224, 224)def load_and_preprocess(img_path):    img = image.load_img(img_path, target_size=IMG_SIZE)    img_arr = image.img_to_array(img) / 255.0    img_arr = np.expand_dims(img_arr, axis=0)    return img, img_arr

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):    grad_model = tf.keras.models.Model(        [model.inputs],        [model.get_layer(last_conv_layer_name).output, model.output]    )    with tf.GradientTape() as tape:        conv_outputs, predictions = grad_model(img_array)        pred_index = tf.argmax(predictions[0])        pred_output = predictions[:, pred_index]    grads = tape.gradient(pred_output, conv_outputs)    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))    conv_outputs = conv_outputs[0]    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)    return heatmap.numpy()

In [ ]:
from google.colab import filesuploaded = files.upload()test_img_path = list(uploaded.keys())[0]  print("Image selected:", test_img_path)

In [ ]:
for i, layer in enumerate(model.layers):    print(i, layer.name)

In [ ]:
for i, layer in enumerate(model.layers[0].layers):    print(i, layer.name)

In [ ]:
base_model = model.layers[0]     last_conv_layer = base_model.get_layer("Conv_1")print("Using last conv layer:", last_conv_layer.name)

In [ ]:
def make_gradcam_heatmap(img_array, model, base_model, last_conv_layer_name):        last_conv_layer = base_model.get_layer(last_conv_layer_name)        grad_model = tf.keras.Model(        inputs=base_model.input,        outputs=[last_conv_layer.output, base_model.output]    )        with tf.GradientTape() as tape:        conv_outputs, predictions = grad_model(img_array)        pred_index = tf.argmax(model.predict(img_array)[0])  # use full model for final class        pred_output = predictions[:, pred_index]    grads = tape.gradient(pred_output, conv_outputs)       pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))        conv_outputs = conv_outputs[0]    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)      heatmap = np.maximum(heatmap, 0)    heatmap /= np.max(heatmap) + 1e-9    return heatmap

In [ ]:
orig_img, img_array = load_and_preprocess(test_img_path)heatmap = make_gradcam_heatmap(    img_array,    model,    base_model,    "Conv_1")plt.figure(figsize=(6,6))plt.imshow(heatmap)plt.title("Grad-CAM Heatmap")plt.axis("off")plt.savefig("gradcam_heatmap.png")plt.show()

In [ ]:
import cv2import numpy as npimport matplotlib.pyplot as pltdef overlay_gradcam(heatmap, original_img, alpha=0.5):       original = np.array(original_img)        heatmap = cv2.resize(heatmap, (original.shape[1], original.shape[0]))       heatmap = np.uint8(255 * heatmap)    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)        overlay = cv2.addWeighted(original, 1 - alpha, heatmap_color, alpha, 0)    return overlayoverlay_img = overlay_gradcam(heatmap, orig_img)plt.figure(figsize=(6,6))plt.imshow(overlay_img)plt.title("Grad-CAM Overlay")plt.axis("off")plt.savefig("gradcam_overlay.png")plt.show()